Andrew Ryan

AndrewRyan@pitt.edu

# Northernlion Live Super Show Analysis

I'm interested in looking at infortion about a Twitch stream, the Northernlion Live Super Show.
The livestream has been airing three times a week since 2013. It features a static and rotating cast, and usually contains three different sections of games played over a three hour period.

A website http://twoandahalfscums.blogspot.co.uk/p/nlss.html contains a text file (NLSS_Dockets) of past NLSS. I turned it into a text file and cleaned it up by removing non-NLSS episodes listed and errors with the dates.

I also am using https://sullygnome.com/channel/Northernlion for further Twitch stream statistics. The data below was gathered from the 365 day data listed on the site, but I have contacted the sites master and have received csv containing information about older streams. This is contained in the FullCSV folder.

Below is some processing of the data into DataFrames and minor analysis of the data, as well as troubleshooting.

In [1]:
import nltk
import pandas as pd
import numpy as np

In [2]:
with open('NLSS_Dockets.txt') as f:
    file = f.read()
shows = file.split('\n\n') #split into every show
shows[:5]

['(August 24, 2017) Nick View (NL, RLS, CS, rob)\nPasspartout, Party Panic, Pinturillo',
 '(August 23, 2017) Nick View (NL, RLS, rob w/ Baer, LGW, HCJ)\nAbsolver, Golf It, Quiplash',
 '(August 21, 2017) Nick View (NL, RLS, JS, rob)\nFire Pro Wrestling World, Ultimate Chicken Horse, Blood Party',
 '(August 17, 2017) Nick View (NL w/ Sin, RLS, LGW, HCJ, Baer)\nGeoguessr, Golf It, Quiplash',
 '(August 16, 2017) Nick View (NL, RLS w/ rob, Baer, LGW, Dan)\nNidhogg 2, Speedrunners, Pinturillo']

Newer shows have Nick View text. Let's get rid of this.

In [3]:
index = 0
for s in shows:
    shows[index] = s.replace(' Nick View', '')
    index+=1
shows[-10:]

['(March 27, 2013) (NL, RLS, JS w/ Ohm)\nDark Souls Invasions, Trivia, Arma III, Ask me anything on Twitter',
 '(March 18, 2013) (NL, RLS, JS)\nDark Souls Invasions, Trivia, Rollercoaster Tycoon, More Dark Souls Invasions, Ask me anything on Twitter',
 '(March 14, 2013) (NL, RLS, JS w/ MALF)\nDark Souls Invasions, Trivia, Worms Revolution, Ask me anything on Twitter',
 '(March 13, 2013) (NL, RLS, JS w/ Ohm)\nDark Souls Invasions, Trivia, The Showdown Effect',
 '(March 11, 2013) (NL, RLS, JS w/ Ohm)\nDark Souls Invasions, Trivia, More Dark Souls Invasions, Arma III, Ask me anything on Twitter',
 '(March 6, 2013) (NL, RLS, JS)\nDark Souls invasions, Trivia, Tomb Raider, Ask me anything on Twitter',
 "(March 4, 2013) (NL, RLS, JS)\nDelver's Drop with Ryan Baker and Ryan Burrell, Dark Souls, Trivia, More Dark Souls, Ask me anything on Twitter",
 '(February 28, 2013) (NL, RLS, JS)\nDark Souls, Trivia, Trials Evolution, Ask me anything on Twitter',
 '(February 27, 2013) (NL, Kate)\nDark Soul

In [4]:
games = []
for s in shows:
    g = s.split('\n')
    games.append(g[1])
games

['Passpartout, Party Panic, Pinturillo',
 'Absolver, Golf It, Quiplash',
 'Fire Pro Wrestling World, Ultimate Chicken Horse, Blood Party',
 'Geoguessr, Golf It, Quiplash',
 'Nidhogg 2, Speedrunners, Pinturillo',
 'Hitman, Ben & Edd Blood Party, London 2012',
 'Ultimate Chicken Horse, Tower Unite, Who Wants To Be A Millionaire, Super Hexagon',
 'Passpartout, Witch It, Quiplash',
 'Afterbirth+, Golf with your Friends, Guesspionage',
 'Tricky Towers, Sonic and All Stars Racing Transformed, London 2012',
 'Ben and Ed - Blood Party, Ball 3D: Soccer Online, Quiplash',
 'Fortnite, Golf It, Gang Beasts',
 'Cannon Brawl, Goldeneye: Source, Ball 3D: Soccer Online',
 'Hitman, Ben & Ed - Blood Party, Tower Unite',
 'Ben and Ed - Blood Party, Golf with Your Friends, Pinturillo',
 "Ben and Edd's Blood Party, London 2012, Quiplash",
 'Passpartout, Ultimate Chicken Horse, Tower Unite',
 'The End Is Nigh, Passpartout, Quiplash',
 'Hitman, (continued), Tower Unite, Guesspionage',
 'Passpartout: The Star

In [5]:
len(games)

588

In [6]:
date_crew = []
for s in shows:
    dc = s.split('\n')[0]
    date_crew.append(dc)
print(date_crew)

['(August 24, 2017) (NL, RLS, CS, rob)', '(August 23, 2017) (NL, RLS, rob w/ Baer, LGW, HCJ)', '(August 21, 2017) (NL, RLS, JS, rob)', '(August 17, 2017) (NL w/ Sin, RLS, LGW, HCJ, Baer)', '(August 16, 2017) (NL, RLS w/ rob, Baer, LGW, Dan)', '(August 14, 2017) (NL, JS, MALF, LGW w/ Baer, HCJ)', '(August 10, 2017) (NL, RLS, rob, LGW)', '(August 7, 2017) (NL, RLS, JS, rob w/ Baer)', '(August 3, 2017) (NL, RLS, CS w/ rob, MALF)', '(August 2, 2017) (NL, RLS, LGW w/ Baer, Kory)', '(July 31, 2017) (NL, RLS, JS, rob w/ Sin, Baer, TB)', '(July 27, 2017) (NL, RLS, CS w/ MALF)', '(July 26, 2017) (NL, RLS w/ LGW, Sin, Baer, Dan)', '(July 24, 2017) (NL, RLS, JS w/ LGW)', '(July 20, 2017) (NL, RLS, CS, LGW w/ Baer)', '(July 19, 2017) (NL, RLS, LGW w/ MALF, rob, Baer)', '(July 13, 2017) (NL, RLS, rob w/ LGW, Baer)', '(July 12, 2017) (NL, RLS, rob w/ Baer)', '(July 10, 2017) part 1, part 2 (NL, RLS, JS w/ rob, Sin)', '(July 6, 2017) (NL, RLS, CS, rob w/ Baer)', '(July 5, 2017) (NL, RLS, rob w/ LGW, 

In [7]:
import re
date = []
crew = []
for entry in date_crew:
    foo = re.search(r'\((.*)\)', entry).group(1)
    d = foo.split(r')')[0]
    date.append(d)
    c = foo.split(r'(')[-1]
    crew.append(c)

In [8]:
date_df = pd.DataFrame(date, columns = ["Date"])
date_df.head()

,Date
0,"August 24, 2017"
1,"August 23, 2017"
2,"August 21, 2017"
3,"August 17, 2017"
4,"August 16, 2017"


In [9]:
games_df = pd.DataFrame(games, columns = ["Docket"])
games_df.head()

,Docket
0,"Passpartout, Party Panic, Pinturillo"
1,"Absolver, Golf It, Quiplash"
2,"Fire Pro Wrestling World, Ultimate Chicken Hor..."
3,"Geoguessr, Golf It, Quiplash"
4,"Nidhogg 2, Speedrunners, Pinturillo"


In [10]:
crew_df = pd.DataFrame(crew, columns = ["Crew"])
crew_df.head()

,Crew
0,"NL, RLS, CS, rob"
1,"NL, RLS, rob w/ Baer, LGW, HCJ"
2,"NL, RLS, JS, rob"
3,"NL w/ Sin, RLS, LGW, HCJ, Baer"
4,"NL, RLS w/ rob, Baer, LGW, Dan"


In [11]:
nlss_df = pd.DataFrame()
nlss_df['Date'] = date_df['Date']
nlss_df['Crew'] = crew_df['Crew']
nlss_df['Docket'] = games_df['Docket']
nlss_df.head()

,Date,Crew,Docket
0,"August 24, 2017","NL, RLS, CS, rob","Passpartout, Party Panic, Pinturillo"
1,"August 23, 2017","NL, RLS, rob w/ Baer, LGW, HCJ","Absolver, Golf It, Quiplash"
2,"August 21, 2017","NL, RLS, JS, rob","Fire Pro Wrestling World, Ultimate Chicken Hor..."
3,"August 17, 2017","NL w/ Sin, RLS, LGW, HCJ, Baer","Geoguessr, Golf It, Quiplash"
4,"August 16, 2017","NL, RLS w/ rob, Baer, LGW, Dan","Nidhogg 2, Speedrunners, Pinturillo"


In [12]:
nlss_df.describe()

,Date,Crew,Docket
count,588,588,588
unique,588,223,565
top,"December 4,2014","NL, RLS, JS","Dark Souls invasions, Trivia, Spelunky"
freq,1,111,3


Removing "(continued)" from games list

In [13]:
improved = []
#For each docket
for d in nlss_df['Docket']:
    #Split docket into list of games
    d = d.split(r',')
    #For each game
    for g in d:
        #If game matches string to remove
        if g == r" (continued)" or g == r" (Continued)":
            #Remove game
            d.remove(g)
    improved.append(d)
nlss_df['Docket'] = improved

In [14]:
nlss_df.head()

,Date,Crew,Docket
0,"August 24, 2017","NL, RLS, CS, rob","[Passpartout, Party Panic, Pinturillo]"
1,"August 23, 2017","NL, RLS, rob w/ Baer, LGW, HCJ","[Absolver, Golf It, Quiplash]"
2,"August 21, 2017","NL, RLS, JS, rob","[Fire Pro Wrestling World, Ultimate Chicken H..."
3,"August 17, 2017","NL w/ Sin, RLS, LGW, HCJ, Baer","[Geoguessr, Golf It, Quiplash]"
4,"August 16, 2017","NL, RLS w/ rob, Baer, LGW, Dan","[Nidhogg 2, Speedrunners, Pinturillo]"


In [15]:
nlss_df['Crew']

0                                  NL, RLS, CS, rob
1                    NL, RLS, rob w/ Baer, LGW, HCJ
2                                  NL, RLS, JS, rob
3                    NL w/ Sin, RLS, LGW, HCJ, Baer
4                    NL, RLS w/ rob, Baer, LGW, Dan
5                    NL, JS, MALF, LGW w/ Baer, HCJ
6                                 NL, RLS, rob, LGW
7                          NL, RLS, JS, rob w/ Baer
8                          NL, RLS, CS w/ rob, MALF
9                        NL, RLS, LGW w/ Baer, Kory
10                NL, RLS, JS, rob w/ Sin, Baer, TB
11                              NL, RLS, CS w/ MALF
12                   NL, RLS w/ LGW, Sin, Baer, Dan
13                               NL, RLS, JS w/ LGW
14                         NL, RLS, CS, LGW w/ Baer
15                  NL, RLS, LGW w/ MALF, rob, Baer
16                        NL, RLS, rob w/ LGW, Baer
17                             NL, RLS, rob w/ Baer
18                          NL, RLS, JS w/ rob, Sin
19          

Want to split on "w/" so each crew member is individual item

In [16]:
improved = []
#For each cast of crew
for e in nlss_df['Crew']:
    #Split cast into list of members
    e = e.split(r',')
    #For each member
    for m in e:
        #If member contains a /w
        if r'w/' in m:
            both = m.split(r'w/')
            e.remove(m)
            e.extend(both)
    improved.append(e)
improved[:20]

[['NL', ' RLS', ' CS', ' rob'],
 ['NL', ' RLS', ' LGW', ' HCJ', ' rob ', ' Baer'],
 ['NL', ' RLS', ' JS', ' rob'],
 [' RLS', ' LGW', ' HCJ', ' Baer', 'NL ', ' Sin'],
 ['NL', ' Baer', ' LGW', ' Dan', ' RLS ', ' rob'],
 ['NL', ' JS', ' MALF', ' HCJ', ' LGW ', ' Baer'],
 ['NL', ' RLS', ' rob', ' LGW'],
 ['NL', ' RLS', ' JS', ' rob ', ' Baer'],
 ['NL', ' RLS', ' MALF', ' CS ', ' rob'],
 ['NL', ' RLS', ' Kory', ' LGW ', ' Baer'],
 ['NL', ' RLS', ' JS', ' Baer', ' TB', ' rob ', ' Sin'],
 ['NL', ' RLS', ' CS ', ' MALF'],
 ['NL', ' Sin', ' Baer', ' Dan', ' RLS ', ' LGW'],
 ['NL', ' RLS', ' JS ', ' LGW'],
 ['NL', ' RLS', ' CS', ' LGW ', ' Baer'],
 ['NL', ' RLS', ' rob', ' Baer', ' LGW ', ' MALF'],
 ['NL', ' RLS', ' Baer', ' rob ', ' LGW'],
 ['NL', ' RLS', ' rob ', ' Baer'],
 ['NL', ' RLS', ' Sin', ' JS ', ' rob'],
 ['NL', ' RLS', ' CS', ' rob ', ' Baer']]

Strip extra spaces

In [17]:
fullstripped = []
for entry in improved:
    stripped = []
    for member in entry:
        member = member.strip(' ')
        stripped.append(member)
    fullstripped.append(stripped)
fullstripped

[['NL', 'RLS', 'CS', 'rob'],
 ['NL', 'RLS', 'LGW', 'HCJ', 'rob', 'Baer'],
 ['NL', 'RLS', 'JS', 'rob'],
 ['RLS', 'LGW', 'HCJ', 'Baer', 'NL', 'Sin'],
 ['NL', 'Baer', 'LGW', 'Dan', 'RLS', 'rob'],
 ['NL', 'JS', 'MALF', 'HCJ', 'LGW', 'Baer'],
 ['NL', 'RLS', 'rob', 'LGW'],
 ['NL', 'RLS', 'JS', 'rob', 'Baer'],
 ['NL', 'RLS', 'MALF', 'CS', 'rob'],
 ['NL', 'RLS', 'Kory', 'LGW', 'Baer'],
 ['NL', 'RLS', 'JS', 'Baer', 'TB', 'rob', 'Sin'],
 ['NL', 'RLS', 'CS', 'MALF'],
 ['NL', 'Sin', 'Baer', 'Dan', 'RLS', 'LGW'],
 ['NL', 'RLS', 'JS', 'LGW'],
 ['NL', 'RLS', 'CS', 'LGW', 'Baer'],
 ['NL', 'RLS', 'rob', 'Baer', 'LGW', 'MALF'],
 ['NL', 'RLS', 'Baer', 'rob', 'LGW'],
 ['NL', 'RLS', 'rob', 'Baer'],
 ['NL', 'RLS', 'Sin', 'JS', 'rob'],
 ['NL', 'RLS', 'CS', 'rob', 'Baer'],
 ['NL', 'RLS', 'Baer', 'rob', 'LGW'],
 ['NL', 'RLS', 'LGW', 'Baer', 'rob', 'Sin'],
 ['NL', 'RLS', 'CS', 'rob', 'JS'],
 ['NL', 'RLS', 'Baer', 'rob', 'LGW'],
 ['NL', 'RLS', 'Kate', 'Baer', 'JS', 'rob'],
 ['rob', 'NL', 'MALF'],
 ['LGW', 'JS', 

Looking better. Let's swap it into our DF.

In [18]:
nlss_df['Crew'] = fullstripped
nlss_df.head()

,Date,Crew,Docket
0,"August 24, 2017","[NL, RLS, CS, rob]","[Passpartout, Party Panic, Pinturillo]"
1,"August 23, 2017","[NL, RLS, LGW, HCJ, rob, Baer]","[Absolver, Golf It, Quiplash]"
2,"August 21, 2017","[NL, RLS, JS, rob]","[Fire Pro Wrestling World, Ultimate Chicken H..."
3,"August 17, 2017","[RLS, LGW, HCJ, Baer, NL, Sin]","[Geoguessr, Golf It, Quiplash]"
4,"August 16, 2017","[NL, Baer, LGW, Dan, RLS, rob]","[Nidhogg 2, Speedrunners, Pinturillo]"


# Adding more stats

File from https://sullygnome.com/channel/Northernlion/365/streams

Can only go back 365 days. Can we create a column for date that matches nlss_df format? If so, we can combine overlapping stats.

In [19]:
import os
import glob
print(os.getcwd())

allFiles = glob.glob("*.csv")
stream_df = pd.DataFrame()
l = []
for foo in allFiles:
    stream_df = pd.read_csv(foo,index_col=None, header=0)
    l.append(stream_df)
stream_df = pd.concat(l)

#stream_df = pd.read_csv(r'StreamStats365.csv')
stream_df

C:\Users\sonofabrat\Documents\Data_Science\Class-Practice-Repo\project_ideas


,Unnamed: 0,Stream start time,End time,Stream length,Avg Viewers,Peak viewers,Followers gained,Followers per hour,Views,Views per hour,Games
0,1,Thursday 21st September 2017 21:15,Friday 22nd September 2017 00:15,13,4636,5390,36,2.77,5984,460.31,GeoGuessr|GeoGuessr|https://static-cdn.jtvnw.n...
1,2,Wednesday 20th September 2017 21:15,Thursday 21st September 2017 00:00,12,5245,5988,49,4.08,4881,406.75,GeoGuessr|GeoGuessr|https://static-cdn.jtvnw.n...
2,3,Tuesday 19th September 2017 20:45,Tuesday 19th September 2017 23:15,11,2718,3319,48,4.36,2160,196.36,Life Is Strange|Life_Is_Strange|https://static...
3,4,Monday 18th September 2017 22:15,Tuesday 19th September 2017 01:15,13,4911,5325,45,3.46,4793,368.69,Ben and Ed: Blood Party|Ben_and_Ed_Blood_Party...
4,5,Sunday 17th September 2017 20:30,Sunday 17th September 2017 23:30,13,3074,3527,97,7.46,3380,260.00,Fallout 3|Fallout_3|https://static-cdn.jtvnw.n...
5,6,Thursday 14th September 2017 21:15,Friday 15th September 2017 00:15,13,4562,5363,51,3.92,4818,370.62,The Binding of Isaac: Afterbirth|The_Binding_o...
6,7,Wednesday 13th September 2017 21:15,Thursday 14th September 2017 00:15,13,4596,5601,48,3.69,6174,474.92,Tooth and Tail|Tooth_and_Tail|https://static-c...
7,8,Tuesday 12th September 2017 20:15,Tuesday 12th September 2017 23:00,12,2557,2980,52,4.33,3881,323.42,Life Is Strange|Life_Is_Strange|https://static...
8,9,Monday 11th September 2017 22:00,Tuesday 12th September 2017 01:15,14,4683,5555,89,6.36,3478,248.43,Rock of Ages II: Bigger & Boulder|Rock_of_Ages...
9,10,Sunday 10th September 2017 20:30,Sunday 10th September 2017 23:30,13,3304,3604,174,13.38,5416,416.62,Fallout 3|Fallout_3|https://static-cdn.jtvnw.n...


In [20]:
formatted = []
order = [1,0,2]
for date in stream_df['Stream start time']:
    dmy = date.split(' ')[1:-1] #Date/Month/Year
    dmy[0] = dmy[0][:-2] #Remove day suffixes
    mdy = [dmy[i] for i in order]
    formatted.append(str(mdy[0] + " " + mdy[1] + ", " + mdy[2]))
formatted[:15]

['September 21, 2017',
 'September 20, 2017',
 'September 19, 2017',
 'September 18, 2017',
 'September 17, 2017',
 'September 14, 2017',
 'September 13, 2017',
 'September 12, 2017',
 'September 11, 2017',
 'September 10, 2017',
 'September 7, 2017',
 'September 6, 2017',
 'September 6, 2017',
 'August 30, 2017',
 'August 29, 2017']

In [21]:
stream_df["Date"] = formatted
stream_df = stream_df.reset_index()
stream_df.index = stream_df["index"]
stream_df = stream_df.drop('index', axis=1)
stream_df

,Unnamed: 0,Stream start time,End time,Stream length,Avg Viewers,Peak viewers,Followers gained,Followers per hour,Views,Views per hour,Games,Date
index,,,,,,,,,,,,
0,1,Thursday 21st September 2017 21:15,Friday 22nd September 2017 00:15,13,4636,5390,36,2.77,5984,460.31,GeoGuessr|GeoGuessr|https://static-cdn.jtvnw.n...,"September 21, 2017"
1,2,Wednesday 20th September 2017 21:15,Thursday 21st September 2017 00:00,12,5245,5988,49,4.08,4881,406.75,GeoGuessr|GeoGuessr|https://static-cdn.jtvnw.n...,"September 20, 2017"
2,3,Tuesday 19th September 2017 20:45,Tuesday 19th September 2017 23:15,11,2718,3319,48,4.36,2160,196.36,Life Is Strange|Life_Is_Strange|https://static...,"September 19, 2017"
3,4,Monday 18th September 2017 22:15,Tuesday 19th September 2017 01:15,13,4911,5325,45,3.46,4793,368.69,Ben and Ed: Blood Party|Ben_and_Ed_Blood_Party...,"September 18, 2017"
4,5,Sunday 17th September 2017 20:30,Sunday 17th September 2017 23:30,13,3074,3527,97,7.46,3380,260.00,Fallout 3|Fallout_3|https://static-cdn.jtvnw.n...,"September 17, 2017"
5,6,Thursday 14th September 2017 21:15,Friday 15th September 2017 00:15,13,4562,5363,51,3.92,4818,370.62,The Binding of Isaac: Afterbirth|The_Binding_o...,"September 14, 2017"
6,7,Wednesday 13th September 2017 21:15,Thursday 14th September 2017 00:15,13,4596,5601,48,3.69,6174,474.92,Tooth and Tail|Tooth_and_Tail|https://static-c...,"September 13, 2017"
7,8,Tuesday 12th September 2017 20:15,Tuesday 12th September 2017 23:00,12,2557,2980,52,4.33,3881,323.42,Life Is Strange|Life_Is_Strange|https://static...,"September 12, 2017"
8,9,Monday 11th September 2017 22:00,Tuesday 12th September 2017 01:15,14,4683,5555,89,6.36,3478,248.43,Rock of Ages II: Bigger & Boulder|Rock_of_Ages...,"September 11, 2017"


In [22]:
combined = nlss_df.merge(stream_df)
#drop eronious columns
combined = combined.drop('Games', 1)
combined = combined.drop('Unnamed: 0', 1)

In [23]:
nlss_df.head()

,Date,Crew,Docket
0,"August 24, 2017","[NL, RLS, CS, rob]","[Passpartout, Party Panic, Pinturillo]"
1,"August 23, 2017","[NL, RLS, LGW, HCJ, rob, Baer]","[Absolver, Golf It, Quiplash]"
2,"August 21, 2017","[NL, RLS, JS, rob]","[Fire Pro Wrestling World, Ultimate Chicken H..."
3,"August 17, 2017","[RLS, LGW, HCJ, Baer, NL, Sin]","[Geoguessr, Golf It, Quiplash]"
4,"August 16, 2017","[NL, Baer, LGW, Dan, RLS, rob]","[Nidhogg 2, Speedrunners, Pinturillo]"


In [24]:
combined.head()

,Date,Crew,Docket,Stream start time,End time,Stream length,Avg Viewers,Peak viewers,Followers gained,Followers per hour,Views,Views per hour
0,"August 24, 2017","[NL, RLS, CS, rob]","[Passpartout, Party Panic, Pinturillo]",Thursday 24th August 2017 21:15,Friday 25th August 2017 00:15,13,4715,5751,91,7.00,5400,415.38
1,"August 23, 2017","[NL, RLS, LGW, HCJ, rob, Baer]","[Absolver, Golf It, Quiplash]",Wednesday 23rd August 2017 21:15,Thursday 24th August 2017 00:15,13,5214,6185,170,13.08,5962,458.62
2,"August 21, 2017","[NL, RLS, JS, rob]","[Fire Pro Wrestling World, Ultimate Chicken H...",Monday 21st August 2017 22:15,Tuesday 22nd August 2017 01:15,13,4797,5347,95,7.31,5065,389.62
3,"August 17, 2017","[RLS, LGW, HCJ, Baer, NL, Sin]","[Geoguessr, Golf It, Quiplash]",Thursday 17th August 2017 21:15,Friday 18th August 2017 00:15,13,5214,6075,108,8.31,5365,412.69
4,"August 16, 2017","[NL, Baer, LGW, Dan, RLS, rob]","[Nidhogg 2, Speedrunners, Pinturillo]",Wednesday 16th August 2017 21:15,Thursday 17th August 2017 00:30,14,4681,5250,116,8.29,5823,415.93


In [25]:
result = pd.concat([nlss_df, combined], axis=1)
#Removes repeat columns
result = result.T.groupby(level=0).first().T
#Reorder columns
result = result[['Date','Crew','Docket','Stream start time','End time','Stream length','Avg Viewers','Peak viewers','Followers gained','Followers per hour','Views','Views per hour']]

In [26]:
nlss_df = result
nlss_df.loc[50]

Date                                      April 19, 2017
Crew                           [NL, RLS, Baer, LGW, rob]
Docket                [Afterbirth+,  Golf It,  Quiplash]
Stream start time        Wednesday 19th April 2017 21:15
End time                  Thursday 20th April 2017 00:15
Stream length                                         13
Avg Viewers                                         5594
Peak viewers                                        6733
Followers gained                                      61
Followers per hour                                  4.69
Views                                               9790
Views per hour                                    753.08
Name: 50, dtype: object

In [27]:
nlss_df[70:85]

,Date,Crew,Docket,Stream start time,End time,Stream length,Avg Viewers,Peak viewers,Followers gained,Followers per hour,Views,Views per hour
70,"February 15, 2017","[NL, RLS, rob, LGW, Mathas]","[Ultimate Chicken Horse, Golf It, For Honor]",Wednesday 15th February 2017 22:15,Thursday 16th February 2017 01:15,13,5306,6401,36,2.77,7061,543.15
71,"February 13, 2017","[NL, LGW, JS, rob]","[Afterbirth+, Scribblenauts, Invisigun Heroes]",Monday 13th February 2017 23:15,Tuesday 14th February 2017 02:15,13,6760,8271,51,3.92,8006,615.85
72,"February 9, 2017","[NL, RLS, rob, Baer, LGW, MALF]","[Afterbirth+, For Honor, Quiplash]",Thursday 9th February 2017 23:15,Friday 10th February 2017 02:15,13,6210,7351,41,3.15,7083,544.85
73,February 8,"[NL, RLS, LGW, MALF, Baer, CS, rob]","[Afterbirth+, Sonic All Star Racing Transform...",Monday 6th February 2017 23:15,Tuesday 7th February 2017 02:15,13,6515,7876,82,6.31,10047,772.85
74,"February 6, 2017","[NL, RLS, JS, MALF]","[Afterbirth+, Disc Jam, Gang Beasts]",Thursday 2nd February 2017 23:15,Friday 3rd February 2017 02:15,13,6095,6954,75,5.77,8344,641.85
75,"February 2, 2017","[NL, RLS, rob, MALF, LGW, Baer]","[Afterbirth+, Sonic All Stars Racing Transfor...",Wednesday 1st February 2017 22:15,Thursday 2nd February 2017 01:00,12,6062,6565,73,6.08,9477,789.75
76,"February 1, 2017","[NL, RLS, rob, LGW]","[Afterbirth+, Ultimate Chicken Horse, Golf W...",Tuesday 31st January 2017 00:15,Tuesday 31st January 2017 03:00,12,7445,8393,78,6.5,7689,640.75
77,"January 31, 2017","[NL, JS, MALF, rob, Sin]","[Afterbirth+, Scribblenauts, Quiplash]",Thursday 26th January 2017 23:15,Friday 27th January 2017 02:00,12,5804,6611,36,3,5792,482.67
78,"January 26, 2017","[NL, MALF, rob, LGW]","[Enter the Gungeon, Disc Jam, Pinturillo]",Wednesday 25th January 2017 22:15,Thursday 26th January 2017 01:00,12,6772,7702,54,4.5,6710,559.17
79,"January 25, 2017","[NL, rob, Dan, Sin, Kate, LGW, MALF]","[Afterbirth+, Ultimate Chicken Horse, Quiplash]",Monday 23rd January 2017 23:15,Tuesday 24th January 2017 01:00,11,7750,9012,92,8.36,12865,1169.55


In [28]:
nlss_df.loc[nlss_df['Date']=="Wednesday 8th February 2017 22:15"]

,Date,Crew,Docket,Stream start time,End time,Stream length,Avg Viewers,Peak viewers,Followers gained,Followers per hour,Views,Views per hour


# Let's Explore

Which show had most peak viewers?

In [29]:
mpv = nlss_df.loc[nlss_df['Peak viewers'].idxmax()]
print("Date:", mpv["Date"])
print("Peak viewers:", mpv['Peak viewers'])
print("Peak percentage:", (mpv['Peak viewers']/mpv['Views'])*100)
print("Total viewers:", mpv['Views'])
print("Games:", mpv['Docket'])
nlss_df.loc[nlss_df['Peak viewers'].idxmax()]

Date: January 4, 2017
Peak viewers: 17249.0
Peak percentage: 49.30116900563066
Total viewers: 34987.0
Games: ['Afterbirth+', ' RedOut (continued)', ' Pinturillo']


Date                                                  January 4, 2017
Crew                                        [NL, RLS, LGW, rob, MALF]
Docket                [Afterbirth+,  RedOut (continued),  Pinturillo]
Stream start time                    Wednesday 4th January 2017 00:30
End time                             Wednesday 4th January 2017 04:30
Stream length                                                      17
Avg Viewers                                                     14806
Peak viewers                                                    17249
Followers gained                                                 1581
Followers per hour                                                 93
Views                                                           34987
Views per hour                                                2058.06
Name: 88, dtype: object

When combining CSVs, data not lining up (Feb8/Feb6)